# Train Notebook

This notebook trains a RandomForestRegressor using the processed data produced by `prepare.py`.
It explains each step, shows key shapes, and saves the trained model plus feature names.


## 1) Load configuration & resolve paths
We search upward for `params.yaml` so the notebook works from `notebooks/`.

In [ ]:

from pathlib import Path
import os, json
import pandas as pd
import yaml

# Locate the repo root by searching upward for params.yaml
def find_repo_root(start: Path | None = None) -> Path:
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "params.yaml").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    raise FileNotFoundError("Could not find 'params.yaml' in parent directories. Start this notebook from your repo.")

ROOT = find_repo_root(Path.cwd())
print(f'Resolved ROOT: {ROOT}')
PARAMS_PATH = ROOT / "params.yaml"

with open(PARAMS_PATH, "r") as f:
    params = yaml.safe_load(f)

DATA_DIR = ROOT / "data"
PROCESSED = DATA_DIR / "processed"
MODELS = ROOT / "models"
REPORTS = ROOT / "reports"
PROCESSED.mkdir(parents=True, exist_ok=True)
MODELS.mkdir(parents=True, exist_ok=True)
REPORTS.mkdir(parents=True, exist_ok=True)

display(params)


## 2) Load processed datasets
We load `X_train.csv`, `X_test.csv`, `y_train.csv`, `y_test.csv` from `data/processed/`.
If the lengths mismatch, the notebook will raise a clear error.

In [ ]:

# Load processed data
X_train = pd.read_csv(PROCESSED / "X_train.csv")
X_test  = pd.read_csv(PROCESSED / "X_test.csv")

def read_y(path: Path, n_expected: int) -> pd.Series:
    y = pd.read_csv(path).iloc[:, 0]
    if len(y) == n_expected:
        return y
    y2 = pd.read_csv(path, header=None).iloc[:, 0]
    if len(y2) == n_expected:
        return y2
    raise ValueError(f"Inconsistent length when reading {path}: got {len(y)} (header) and {len(y2)} (no header), expected {n_expected}.")

y_train = read_y(PROCESSED / "y_train.csv", len(X_train))
y_test  = read_y(PROCESSED / "y_test.csv", len(X_test))

print("Shapes:")
print(" X_train:", X_train.shape, " X_test:", X_test.shape)
print(" y_train:", y_train.shape, " y_test:", y_test.shape)

display(X_train.head(5))


## 3) Configure and train the model
We use parameters from `params.yaml` under the `model:` section.

In [ ]:

from sklearn.ensemble import RandomForestRegressor

model_cfg = params.get("model", {}) or {}
n_estimators = int(model_cfg.get("n_estimators", 200))
max_depth = model_cfg.get("max_depth", None)
max_depth = None if (max_depth in (None, "None")) else int(max_depth)
n_jobs = int(model_cfg.get("n_jobs", -1))
random_state = int(model_cfg.get("random_state", 42))

rf = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    n_jobs=n_jobs,
    random_state=random_state,
)
rf.fit(X_train, y_train)

print("Model trained.")


## 4) Save artifacts
We save the model to `models/model.joblib` and the feature names to `models/feature_names.json`.

In [ ]:

from joblib import dump
feature_names = list(X_train.columns)

model_path = MODELS / "model.joblib"
dump(rf, model_path)

(MODELS / "feature_names.json").write_text(json.dumps(feature_names, indent=2))

print(f"Saved model to: {model_path}")
print(f"Saved feature names to: {MODELS / 'feature_names.json'}")


## 5) Quick sanity visual (optional)
Basic bar chart of top feature importances. If the model has no `feature_importances_`, the cell will skip plotting.

In [ ]:

import matplotlib.pyplot as plt

if hasattr(rf, "feature_importances_"):
    importances = pd.Series(rf.feature_importances_, index=feature_names).sort_values(ascending=False).head(10)
    print("Top 10 feature importances:")
    display(importances.to_frame("importance"))
    plt.figure()
    importances.sort_values().plot(kind="barh")
    plt.title("Top 10 Feature Importances")
    plt.tight_layout()
    plt.show()
else:
    print("Model has no feature_importances_. Skipping plot.")
